### 0. Import libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time, random
from datetime import datetime
from urllib.parse import urljoin

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36" 
}

### 1. Stanovi

In [2]:
BASE = "https://www.oglasi.rs"
url = "https://www.oglasi.rs/nekretnine/prodaja-stanova?p=1"

def get_next_url(soup):
    for li in soup.select("ul.pagination.hidden-xs li"):
        a = li.find("a", href=True)
        if a and "sledeća" in a.get_text(strip=True).lower() and a["href"] != "#":
            return urljoin(BASE, a["href"])
    a = soup.select_one("ul.pager.visible-xs li.next a[href]")
    if a and "sledeća" in a.get_text(strip=True).lower():
        return urljoin(BASE, a["href"])
    return None

In [3]:
rows = []

while url:
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    doc = BeautifulSoup(r.text, "html.parser")

    ads = doc.find_all("article", {"itemprop": "itemListElement"})

    for ad in ads:
        row = {}

        row['naslov'] = ad.find('h2', {"itemprop": "name"}).text
        row['link'] = ad.find('a').get('href')

        lok = ad.find_all("a", {"itemprop": "category"})
        row['lokacija'] = " / ".join([l.get_text(strip=True) for l in lok]) if lok else None

        info_divs = ad.select("div.col-sm-6")
        for div in info_divs:
            text = div.get_text(strip=True)
            value = div.find("strong").get_text(strip=True) if div.find("strong") else None

            if "Sobnost" in text:
                row["sobnost"] = value
            elif "Kvadratura" in text:
                row["kvadratura"] = value
            elif "Nivo u zgradi" in text:
                row["sprat"] = value

        price_tag = ad.find(class_="text-price")
        if price_tag:
            cena_raw = price_tag.get_text(" ", strip=True)
            row['cena'] = cena_raw.replace("\xa0", " ").strip()
        else:
            muted = ad.select_one("span.text-muted strong")
            if muted and "cena nije navedena" in muted.get_text(strip=True).lower():
                row['cena'] = "Cena nije navedena"
            else:
                row['cena'] = None

        try:
            row['opis'] = ad.find('p', {"itemprop": "description"}).text
        except:
            row['opis'] = None

        try:
            time_tag = ad.find("time")
            row["obnovljen"] = time_tag.get_text(strip=True) if time_tag else None
        except:
            row["obnovljen"] = None
        
        try:
            ag_tag = ad.find(class_="visible-sm") or ad.find("cite") or ad.find("small")
            row["agencija"] = ag_tag.get_text(strip=True) if ag_tag else None
        except:
            row["agencija"] = None

        img = ad.find("img")
        if img:
            row['slika'] = img.get("src")
        else:
            row['slika'] = None

        rows.append(row)

    next_url = get_next_url(doc)
    if not next_url:
        break
    url = next_url
    time.sleep(random.uniform(0.5, 3.0))

In [4]:
print(len(rows))

41890


In [5]:
stanovi = pd.DataFrame(rows)
stanovi

,naslov,link,lokacija,sobnost,kvadratura,sprat,cena,opis,obnovljen,agencija,slika
0,Trosoban stan u izgradnji na Salajci,/nekretnine/prodaja-stanova/03-3499780/trosoba...,Nekretnine / Prodaja stanova / Novi Sad / Salajka,Trosoban,76m2,1. sprat,"194.410,00 EUR",Predstavljamo Vam novu izgradnju u jednom od n...,29.08.2025.,Prodaja Nekretnine Reg. br. 711,https://media.oglasi.rs/5f47f9b2-6f41-4df1-93d...
1,Trosoban stan u izgradnji na Salajci,/nekretnine/prodaja-stanova/03-3499830/trosoba...,Nekretnine / Prodaja stanova / Novi Sad / Salajka,Trosoban,76m2,2. sprat,"194.410,00 EUR",Predstavljamo Vam novu izgradnju u jednom od n...,29.08.2025.,Prodaja Nekretnine Reg. br. 711,https://media.oglasi.rs/cf359c9d-3efa-443a-9f8...
2,Dvosoban stan u blizini Železničke stanice,/nekretnine/prodaja-stanova/03-3428006/dvosoba...,Nekretnine / Prodaja stanova / Novi Sad / Žele...,Dvosoban,39m2,5. sprat,"122.890,00 EUR",Novogradnja na izuzetno traženoj lokaciji. Bli...,29.08.2025.,Prodaja Nekretnine Reg. br. 711,https://media.oglasi.rs/bd98acb0-1ead-485b-b7c...
3,Odlican stan na Spensu 49m2,/nekretnine/prodaja-stanova/03-3567619/odlican...,Nekretnine / Prodaja stanova / Novi Sad / Cent...,Dvosoban,49m2,3. sprat,"149.350,00 EUR","Predstavljamo vam odličan, uknjižen dvosoban s...",29.08.2025.,Prodaja Nekretnine Reg. br. 711,https://media.oglasi.rs/69901fa9-e3c8-40dc-9b9...
4,Dvoiposoban stan na Adicama!,/nekretnine/prodaja-stanova/03-3113253/dvoipos...,Nekretnine / Prodaja stanova / Novi Sad / Adice,Dvoiposoban,48m2,2. sprat,"95.930,00 EUR",Izgradnja vrhunskog kvaliteta. Cena je sa urač...,29.08.2025.,PLAY NEKRETNINE NS-Group nekretnine doo. Novi ...,https://media.oglasi.rs/28d8d3ff-2bb8-4591-af0...
...,...,...,...,...,...,...,...,...,...,...,...
41885,Uknjižen stan - Banovo brdo ID#3780,/nekretnine/prodaja-stanova/03-3426924/uknjize...,Nekretnine / Prodaja stanova / Beograd / Čukar...,Trosoban,106m2,Visoko prizemlje,"295.000,00 EUR","Uknjižen stan, odmah useljiv, sređen, stavljen...",30.07.2025.,Ivela 011 Reg. br. 391,https://media.oglasi.rs/7d02fe22-9a9e-4cd5-a1b...
41886,Uknjižena garsonjera ID#3650,/nekretnine/prodaja-stanova/03-3270249/uknjize...,Nekretnine / Prodaja stanova / Beograd / Savsk...,Garsonjera,18m2,1. sprat,"97.000,00 EUR",Odlična garsonjera u kvalitetnoj zgradi starij...,30.07.2025.,Ivela 011 Reg. br. 391,https://media.oglasi.rs/ca25d596-d5e9-4b5c-a40...
41887,Useljiv stan - Petlovo brdo ID#3614,/nekretnine/prodaja-stanova/03-3233106/useljiv...,Nekretnine / Prodaja stanova / Beograd / Rakov...,Dvoiposoban,63m2,5. sprat,"90.000,00 EUR","Useljiv stan, sa 2 spavaće sobe.\nGrejanje na ...",30.07.2025.,Ivela 011 Reg. br. 391,https://media.oglasi.rs/1f4b479a-a768-457a-942...
41888,Renoviran stan - kod Maksija ID#3916,/nekretnine/prodaja-stanova/03-3585789/renovir...,Nekretnine / Prodaja stanova / Beograd / Voždo...,Dvoiposoban,76m2,3. sprat,"187.000,00 EUR","Odličan stan, renoviran, izvorno 2.5. \nSvetao...",30.07.2025.,Ivela 011 Reg. br. 391,https://media.oglasi.rs/04c1c2d6-d99f-4609-b35...


### 2. Kuće

In [6]:
BASE = "https://www.oglasi.rs"
url = "https://www.oglasi.rs/nekretnine/prodaja-kuca?p=1"

def get_next_url(soup):
    for li in soup.select("ul.pagination.hidden-xs li"):
        a = li.find("a", href=True)
        if a and "sledeća" in a.get_text(strip=True).lower() and a["href"] != "#":
            return urljoin(BASE, a["href"])
    a = soup.select_one("ul.pager.visible-xs li.next a[href]")
    if a and "sledeća" in a.get_text(strip=True).lower():
        return urljoin(BASE, a["href"])
    return None

In [7]:
rows = []

while url:
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    doc = BeautifulSoup(r.text, "html.parser")

    ads = doc.find_all("article", {"itemprop": "itemListElement"})

    for ad in ads:
        row = {}

        row['naslov'] = ad.find('h2', {"itemprop": "name"}).text
        row['link'] = ad.find('a').get('href')

        lok = ad.find_all("a", {"itemprop": "category"})
        row['lokacija'] = " / ".join([l.get_text(strip=True) for l in lok]) if lok else None

        info_divs = ad.select("div.col-sm-6")
        for div in info_divs:
            text = div.get_text(strip=True)
            value = div.find("strong").get_text(strip=True) if div.find("strong") else None

            if "Sobnost" in text:
                row["sobnost"] = value
            elif "Kvadratura" in text:
                row["kvadratura"] = value
            elif "Nivo u zgradi" in text:
                row["sprat"] = value

        price_tag = ad.find(class_="text-price")
        if price_tag:
            cena_raw = price_tag.get_text(" ", strip=True)
            row['cena'] = cena_raw.replace("\xa0", " ").strip()
        else:
            muted = ad.select_one("span.text-muted strong")
            if muted and "cena nije navedena" in muted.get_text(strip=True).lower():
                row['cena'] = "Cena nije navedena"
            else:
                row['cena'] = None

        try:
            row['opis'] = ad.find('p', {"itemprop": "description"}).text
        except:
            row['opis'] = None

        try:
            time_tag = ad.find("time")
            row["obnovljen"] = time_tag.get_text(strip=True) if time_tag else None
        except:
            row["obnovljen"] = None
        
        try:
            ag_tag = ad.find(class_="visible-sm") or ad.find("cite") or ad.find("small")
            row["agencija"] = ag_tag.get_text(strip=True) if ag_tag else None
        except:
            row["agencija"] = None

        img = ad.find("img")
        if img:
            row['slika'] = img.get("src")
        else:
            row['slika'] = None

        rows.append(row)

    next_url = get_next_url(doc)
    if not next_url:
        break
    url = next_url
    time.sleep(random.uniform(0.5, 3.0))

In [8]:
print(len(rows))

12549


In [9]:
kuce = pd.DataFrame(rows)
kuce

,naslov,link,lokacija,kvadratura,cena,opis,obnovljen,agencija,slika
0,"Rakovac,kuća",/nekretnine/prodaja-kuca/03-3608282/rakovac-kuca,Nekretnine / Prodaja kuća / Rakovac,104m2,"149.000,00 EUR",Predstavljamo vam sjajnu priliku za porodični ...,29.08.2025.,Grad nekretnine NS-Group nekretnine doo Novi S...,/bundles/oglasicommon/images/premium.png
1,***Odlična kuća Beočin 129m2***,/nekretnine/prodaja-kuca/03-3560270/odlicna-ku...,Nekretnine / Prodaja kuća / Čerević,129m2,"123.600,00 EUR","Predstavljamo vam uknjiženu spratnu kuću, povr...",29.08.2025.,Grad nekretnine NS-Group nekretnine doo Novi S...,/bundles/oglasicommon/images/premium.png
2,Fantastična kompletno opremljena kuća! SAVRŠENA!,/nekretnine/prodaja-kuca/03-3574905/fantasticn...,Nekretnine / Prodaja kuća / Novi Sad / Telep,74m2,"299.730,00 EUR","Fantastična ponuda, prelepa kuća koja se može ...",29.08.2025.,Info nekretnine NS-Group nekretnine doo Novi S...,/bundles/oglasicommon/images/premium.png
3,Za investitore,/nekretnine/prodaja-kuca/03-3579825/za-investi...,Nekretnine / Prodaja kuća / Petrovaradin / Sta...,140m2,"412.000,00 EUR",Fantastičan plac pogodan za gradnju višeporodi...,29.08.2025.,Info nekretnine NS-Group nekretnine doo Novi S...,/bundles/oglasicommon/images/premium.png
4,"Odlična vikendica u Vrdniku od 54 m2, u stvarn...",/nekretnine/prodaja-kuca/03-3557328/odlicna-vi...,Nekretnine / Prodaja kuća / Vrdnik,54m2,"74.270,00 EUR","Odlična vikendica u Vrdniku od 54 m2, u stvar...",29.08.2025.,ROYAL NEKRETNINE NS-Group nekretnine doo Novi ...,/bundles/oglasicommon/images/premium.png
...,...,...,...,...,...,...,...,...,...
12544,Odlična investicija,/nekretnine/prodaja-kuca/03-2905047/odlicna-in...,Nekretnine / Prodaja kuća / Pančevo,348m2,"158.000,00 EUR","Prostrana uknjižena kuća novije gradnje, sprat...",30.07.2025.,MEGASTAN Reg. br. 298,https://media.oglasi.rs/c01c50c4-cf35-4bed-b8f...
12545,"Margita, blizina centra",/nekretnine/prodaja-kuca/03-2905048/margita-bl...,Nekretnine / Prodaja kuća / Pančevo,101m2,"140.000,00 EUR","Kuća uknjižene površine 48m2, stvarne površine...",30.07.2025.,MEGASTAN Reg. br. 298,https://media.oglasi.rs/fa40721d-97b2-4556-b7e...
12546,Borča,/nekretnine/prodaja-kuca/03-2743685/borca,Nekretnine / Prodaja kuća / Beograd / Palilula...,239m2,"290.000,00 EUR","Porodična kuća na odličnoj lokaciji, u blizini...",30.07.2025.,MEGASTAN Reg. br. 298,https://media.oglasi.rs/ca2df19b-6fc8-43b8-8bf...
12547,Idealno za beg iz grada,/nekretnine/prodaja-kuca/03-3585813/idealno-za...,Nekretnine / Prodaja kuća / Crepaja,220m2,"110.000,00 EUR","Crepaja, živopisno selo bogate tradicije, nala...",30.07.2025.,MEGASTAN Reg. br. 298,https://media.oglasi.rs/1e17769d-2a03-4643-adb...


### 3. Lokal, poslovni prostor

In [10]:
BASE = "https://www.oglasi.rs"
url = "https://www.oglasi.rs/nekretnine/prodaja-poslovnog-prostora?p=1"

def get_next_url(soup):
    for li in soup.select("ul.pagination.hidden-xs li"):
        a = li.find("a", href=True)
        if a and "sledeća" in a.get_text(strip=True).lower() and a["href"] != "#":
            return urljoin(BASE, a["href"])
    a = soup.select_one("ul.pager.visible-xs li.next a[href]")
    if a and "sledeća" in a.get_text(strip=True).lower():
        return urljoin(BASE, a["href"])
    return None

In [11]:
rows = []

while url:
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    doc = BeautifulSoup(r.text, "html.parser")

    ads = doc.find_all("article", {"itemprop": "itemListElement"})

    for ad in ads:
        row = {}

        row['naslov'] = ad.find('h2', {"itemprop": "name"}).text
        row['link'] = ad.find('a').get('href')

        lok = ad.find_all("a", {"itemprop": "category"})
        row['lokacija'] = " / ".join([l.get_text(strip=True) for l in lok]) if lok else None

        info_divs = ad.select("div.col-sm-6")
        for div in info_divs:
            text = div.get_text(strip=True)
            value = div.find("strong").get_text(strip=True) if div.find("strong") else None

            if "Sobnost" in text:
                row["sobnost"] = value
            elif "Kvadratura" in text:
                row["kvadratura"] = value
            elif "Nivo u zgradi" in text:
                row["sprat"] = value

        price_tag = ad.find(class_="text-price")
        if price_tag:
            cena_raw = price_tag.get_text(" ", strip=True)
            row['cena'] = cena_raw.replace("\xa0", " ").strip()
        else:
            muted = ad.select_one("span.text-muted strong")
            if muted and "cena nije navedena" in muted.get_text(strip=True).lower():
                row['cena'] = "Cena nije navedena"
            else:
                row['cena'] = None

        try:
            row['opis'] = ad.find('p', {"itemprop": "description"}).text
        except:
            row['opis'] = None

        try:
            time_tag = ad.find("time")
            row["obnovljen"] = time_tag.get_text(strip=True) if time_tag else None
        except:
            row["obnovljen"] = None
        
        try:
            ag_tag = ad.find(class_="visible-sm") or ad.find("cite") or ad.find("small")
            row["agencija"] = ag_tag.get_text(strip=True) if ag_tag else None
        except:
            row["agencija"] = None

        img = ad.find("img")
        if img:
            row['slika'] = img.get("src")
        else:
            row['slika'] = None

        rows.append(row)

    next_url = get_next_url(doc)
    if not next_url:
        break
    url = next_url
    time.sleep(random.uniform(0.5, 3.0))

In [12]:
print(len(rows))

2927


In [13]:
lokali = pd.DataFrame(rows)

### 4. Vikendice

In [14]:
BASE = "https://www.oglasi.rs"
url = "https://www.oglasi.rs/nekretnine/prodaja-vikendica?p=1"

def get_next_url(soup):
    for li in soup.select("ul.pagination.hidden-xs li"):
        a = li.find("a", href=True)
        if a and "sledeća" in a.get_text(strip=True).lower() and a["href"] != "#":
            return urljoin(BASE, a["href"])
    a = soup.select_one("ul.pager.visible-xs li.next a[href]")
    if a and "sledeća" in a.get_text(strip=True).lower():
        return urljoin(BASE, a["href"])
    return None

In [15]:
rows = []

while url:
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    doc = BeautifulSoup(r.text, "html.parser")

    ads = doc.find_all("article", {"itemprop": "itemListElement"})

    for ad in ads:
        row = {}

        row['naslov'] = ad.find('h2', {"itemprop": "name"}).text
        row['link'] = ad.find('a').get('href')

        lok = ad.find_all("a", {"itemprop": "category"})
        row['lokacija'] = " / ".join([l.get_text(strip=True) for l in lok]) if lok else None

        info_divs = ad.select("div.col-sm-6")
        for div in info_divs:
            text = div.get_text(strip=True)
            value = div.find("strong").get_text(strip=True) if div.find("strong") else None

            if "Sobnost" in text:
                row["sobnost"] = value
            elif "Kvadratura" in text:
                row["kvadratura"] = value
            elif "Nivo u zgradi" in text:
                row["sprat"] = value

        price_tag = ad.find(class_="text-price")
        if price_tag:
            cena_raw = price_tag.get_text(" ", strip=True)
            row['cena'] = cena_raw.replace("\xa0", " ").strip()
        else:
            muted = ad.select_one("span.text-muted strong")
            if muted and "cena nije navedena" in muted.get_text(strip=True).lower():
                row['cena'] = "Cena nije navedena"
            else:
                row['cena'] = None

        try:
            row['opis'] = ad.find('p', {"itemprop": "description"}).text
        except:
            row['opis'] = None

        try:
            time_tag = ad.find("time")
            row["obnovljen"] = time_tag.get_text(strip=True) if time_tag else None
        except:
            row["obnovljen"] = None
        
        try:
            ag_tag = ad.find(class_="visible-sm") or ad.find("cite") or ad.find("small")
            row["agencija"] = ag_tag.get_text(strip=True) if ag_tag else None
        except:
            row["agencija"] = None

        img = ad.find("img")
        if img:
            row['slika'] = img.get("src")
        else:
            row['slika'] = None

        rows.append(row)

    next_url = get_next_url(doc)
    if not next_url:
        break
    url = next_url
    time.sleep(random.uniform(0.5, 3.0))

In [16]:
print(len(rows))

751


In [17]:
vikendice = pd.DataFrame(rows)
vikendice

,naslov,link,lokacija,kvadratura,cena,opis,obnovljen,agencija,slika
0,Vikendica u Čereviću!,/nekretnine/prodaja-vikendica/03-3478541/viken...,Nekretnine / Prodaja vikendica / Čerević,132m2,"127.310,00 EUR",Samo kod nas u ponudi! Predstavljamo Vam viken...,29.08.2025.,TRG nekretnine NS-Group nekretnine doo. Novi S...,/bundles/oglasicommon/images/premium.png
1,Vikendica na prodaju u Čortanovcima – savršena...,/nekretnine/prodaja-vikendica/03-3558625/viken...,Nekretnine / Prodaja vikendica / Čortanovci,108m2,"59.740,00 EUR",Vikendica u srcu Fruške gore pruža mirno utoči...,27.08.2025.,DVA ZVONA 1963 NEKRETNINE Reg. br. 1472,/bundles/oglasicommon/images/top.png
2,Prelepa vikendica sa voćnjakom - BANSTOL - 70m2,/nekretnine/prodaja-vikendica/03-3524674/prele...,Nekretnine / Prodaja vikendica / Banstol,70m2,"360.500,00 EUR",Predstavljamo Vam odličnu vikendicu na prelepo...,29.08.2025.,ART nekretnine NS-Group Upisan U Reg. Pos. Br...,https://media.oglasi.rs/03c42b41-3fe3-4d9d-973...
3,Prodaje se plac sa vikend kućom na Popovici -70m2,/nekretnine/prodaja-vikendica/03-3588788/proda...,Nekretnine / Prodaja vikendica / Sremska Kamen...,70m2,"85.000,00 EUR",Prodaje se plac sa vikend kućom na Popovici u ...,29.08.2025.,ART nekretnine NS-Group Upisan U Reg. Pos. Br...,https://media.oglasi.rs/06a6dc62-9c64-430c-881...
4,Vikendica sa izvanrednim placem na Karagaci!,/nekretnine/prodaja-vikendica/03-3587701/viken...,Nekretnine / Prodaja vikendica / Sremska Kamen...,40m2,"147.910,00 EUR",Predstavljamo Vam fantastičnu vikedn kuću na o...,29.08.2025.,TRG nekretnine NS-Group nekretnine doo. Novi S...,https://media.oglasi.rs/0c1d36a6-2ddd-4659-b06...
...,...,...,...,...,...,...,...,...,...
746,Plac sa vikendicom na Karagači,/nekretnine/prodaja-vikendica/03-3084898/plac-...,Nekretnine / Prodaja vikendica / Petrovaradin ...,136m2,"135.000,00 EUR","Prodaje se vikendica na Karagači , korisne pov...",31.07.2025.,Dragana,https://media.oglasi.rs/06c8ae2d-c30d-428a-a1b...
747,Prodajem jedinstven plac i vikendicu NA SAMOM ...,/nekretnine/prodaja-vikendica/03-3585409/proda...,Nekretnine / Prodaja vikendica / Neštin,73m2,"153.000,00 EUR",Prodajem jedinstvenu i predivnu vikendicu na s...,30.07.2025.,xx,https://media.oglasi.rs/645447fa-38eb-467f-bef...
748,Kuća u Krčedinu - prirodni raj pored Dunava,/nekretnine/prodaja-vikendica/03-3545768/kuca-...,Nekretnine / Prodaja vikendica / Krčedin,130m2,"149.300,00 EUR","Prodaje se prelepa kuća u Krčedinu, smeštena u...",30.07.2025.,kuce,https://media.oglasi.rs/a679fe75-46af-41c3-941...
749,Na prodaju vikendica i plac sa izuzetnim poten...,/nekretnine/prodaja-vikendica/03-3291540/na-pr...,Nekretnine / Prodaja vikendica / Sremski Karlovci,40m2,"122.570,00 EUR",Na prodaju vikendica i plac sa izuzetnim poten...,30.07.2025.,Bulevar nekretnine NS-Group nekretnine doo Nov...,https://media.oglasi.rs/0259f0d4-6751-4c21-850...


### 5. Merge

In [18]:
oglasi = pd.concat([vikendice, lokali, kuce, stanovi], ignore_index=True)
oglasi.head()

,naslov,link,lokacija,kvadratura,cena,opis,obnovljen,agencija,slika,sprat,sobnost
0,Vikendica u Čereviću!,/nekretnine/prodaja-vikendica/03-3478541/viken...,Nekretnine / Prodaja vikendica / Čerević,132m2,"127.310,00 EUR",Samo kod nas u ponudi! Predstavljamo Vam viken...,29.08.2025.,TRG nekretnine NS-Group nekretnine doo. Novi S...,/bundles/oglasicommon/images/premium.png,NaN,NaN
1,Vikendica na prodaju u Čortanovcima – savršena...,/nekretnine/prodaja-vikendica/03-3558625/viken...,Nekretnine / Prodaja vikendica / Čortanovci,108m2,"59.740,00 EUR",Vikendica u srcu Fruške gore pruža mirno utoči...,27.08.2025.,DVA ZVONA 1963 NEKRETNINE Reg. br. 1472,/bundles/oglasicommon/images/top.png,NaN,NaN
2,Prelepa vikendica sa voćnjakom - BANSTOL - 70m2,/nekretnine/prodaja-vikendica/03-3524674/prele...,Nekretnine / Prodaja vikendica / Banstol,70m2,"360.500,00 EUR",Predstavljamo Vam odličnu vikendicu na prelepo...,29.08.2025.,ART nekretnine NS-Group Upisan U Reg. Pos. Br...,https://media.oglasi.rs/03c42b41-3fe3-4d9d-973...,NaN,NaN
3,Prodaje se plac sa vikend kućom na Popovici -70m2,/nekretnine/prodaja-vikendica/03-3588788/proda...,Nekretnine / Prodaja vikendica / Sremska Kamen...,70m2,"85.000,00 EUR",Prodaje se plac sa vikend kućom na Popovici u ...,29.08.2025.,ART nekretnine NS-Group Upisan U Reg. Pos. Br...,https://media.oglasi.rs/06a6dc62-9c64-430c-881...,NaN,NaN
4,Vikendica sa izvanrednim placem na Karagaci!,/nekretnine/prodaja-vikendica/03-3587701/viken...,Nekretnine / Prodaja vikendica / Sremska Kamen...,40m2,"147.910,00 EUR",Predstavljamo Vam fantastičnu vikedn kuću na o...,29.08.2025.,TRG nekretnine NS-Group nekretnine doo. Novi S...,https://media.oglasi.rs/0c1d36a6-2ddd-4659-b06...,NaN,NaN


In [19]:
timestamp = datetime.now().strftime('%Y-%m-%d')
filename = f"prodaja {timestamp}.csv"
oglasi.to_csv(filename, index=False)